In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import csv as csv

#Add website with https and no / at the end
website ="https://www.infobae.com"

In [ ]:
def is_available(e):
    if e:
        return e.text.strip()
    else:
        return None
def get_home_articles(url:str) -> list:
  article = requests.get(url)
  container = bs(article.content, "html.parser")
  body = container.find("div", class_="homepage-layout")
  #urls = body.find_all("a")
  article_list = {"list":[]}
  counter = 0
    #print(card.a.get("href"))
  for link in body.find_all("a", class_="headline-link"):
    item = {}
    if(is_available(link.h2)!=None):
      item["title"]  = link.h2.text
      item["url"]  = url + link.get("href")
      article_list["list"].append(item)
       
  #print(article_list["list"]) 
  return article_list["list"]

#uncomment for testing
#get_home_articles(website)

In [ ]:

class retrieve_news:
    def __init__(self, url:str):
        article = requests.get(url)
        self.soup = bs(article.content, "html.parser")
        self.body = self.get_body()        
        self.title = self.get_title()
        

    def get_body(self) -> list:
        body = self.soup.find(class_="body-article")
        if(body != None):
            content=""
            for p in body.find_all("p"):
                content = content + p.text
                content = content + "\n"     
            #return [p.text for p in body.find_all("p")]
            return content
        else:
            return None
    
    def get_title(self) -> str:
        return self.soup.find(class_="article-headline").text

In [ ]:
articles = get_home_articles(website)
article = retrieve_news(articles[0]["url"])
row_list = []
#col_names = ["TITLE", "URL", "BODY","SECTION"]
for i in range(len(articles)):
    #print(articles[i]["url"])
    article = retrieve_news(articles[i]["url"])
    row = [article.title, articles[i]["url"], article.body, "home"]
    row_list.append(row)

In [ ]:
filename = 'infobae_news.csv'
with open(filename, 'a+', newline='') as f:
    fields = ['TITLE', 'URL',"BODY","SECTION"]
    writer = csv.writer(f)

    #moving file pointer at the start of the file
    f.seek(0)
    existing_lines = csv.reader(f)
    
    #finding no of lines in the file
    count = 0
    counter = 0
    for line in existing_lines:
        count+=1
        break
    
    if count>0:
        for row_item in row_list:
            flag=0
            #moving file pointer to the start of the file
            f.seek(0)

            #checking if row_item[1] is present in the second  column (URL) of csv file
            for line in existing_lines:
                if row_item[1] in line[1]:
                    flag=1 

            #if row_item[1] is not found                   
            if flag==0:
                writer.writerow(row_item)
                counter += 1
                print(str(counter) + ": "+ row_item[1])
    #if file is empty write the dictionary contents into the csv
    else:
        for row_item in row_list:
            writer.writerow(row_item)